In [1]:
%matplotlib inline

import numpy as np
from pylab import *
from scipy.io import wavfile
import matplotlib.pyplot as plt
from scipy import interpolate
from os import listdir
import librosa
import tensorflow as tf
from matplotlib.pyplot import specgram

In [2]:
'''
compileAnimals = f(compileData, addResponse)
compileData = f(changeSampleRate, dataSlice)
'''


def changeSampleRate(array, rate, new_rate = 166600):
    '''
    Returns signal converted to new sample rate
    
    '''
    
    length = len(array)
    
    x = np.arange(0, length)
    y = array[0:length]
    f = interpolate.interp1d(x, y, kind = 'nearest')
    
    step = rate/float(new_rate)
    
    xnew = np.arange(0, length - 1, step)
    ynew = f(xnew)

    return ynew

def dataSlice(array, slicelength = 166600):
    '''
    input array and desired length of slices
    output list of data slices
    '''
    
    data = []
    slices = len(array)//slicelength
    
    
    for index in np.arange(slices):
        start = index*slicelength
        end = start + slicelength
        datum = array[start:end]
        data.append(datum)
    
    return np.array(data)

def compileData(path):
    files = listdir(path)
    data = []
    for f in files:
        X,sr = librosa.load(path + f)
        new_signal = changeSampleRate(X, sr)
        sliced_signals = dataSlice(new_signal)

        if sliced_signals.shape == (0,): # some signals are too short, remove them so 
            pass                         # there isn't an error in np.vstack below
        else:
            mfccs = librosa.feature.mfcc(y=sliced_signals, sr=sr, n_mfcc=40)
            data.append(mfccs)
            del sliced_signals


    data_numpified = np.vstack(tuple(data))
    del data

    return data_numpified

def addResponse(array, label):
    '''
    Adds responses to data matrix
    '''
    column = np.zeros((array.shape[0],1))
    
    column.fill(label)
    
    #print(array.shape, column.shape)
    array_new = np.hstack((array, column))
    del array, column
    
    return array_new

def compileAnimals(feed_dict):
    
    data = []
    for animal in feed_dict.keys():
    
        array = compileData(animal)
        array_new = addResponse(array, feed_dict[animal])

        data.append(array_new)

        del array, array_new

    data = tuple(data)
    database = np.vstack(data)
    
    return database

In [3]:
feed_dict = {'./BD15L/':0,
             './BD6A/':1,
             './BD6B/':2,
             './BD15C/':3
            }

database = compileAnimals(feed_dict)

ParameterError: Invalid shape for monophonic audio: ndim=2, shape=(1, 166600)

In [ ]:
database[database[:,-1] == 2].shape

In [ ]:
np.save('database_mfcc.npy', database)

In [ ]:
np.load('database.npy')

In [ ]:
database